In [1]:
import pandas as pd
import pandas_ta as ta
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy import stats

In [2]:
df = pd.read_csv ('bitcoin.csv')

df.set_index('Date', inplace=True)

df.index = pd.to_datetime(df.index)

df = df.drop('Volume', axis=1)

df = df.rename (columns= {'Adj Close' : 'close' })  

df ['EMA'] = ta.ema (df.close, length = 20)

df.tail ()

,close,EMA
Date,,
2024-03-27,69455.343750,66638.495956
2024-03-28,70744.953125,67029.587115
2024-03-29,69892.828125,67302.276735
2024-03-30,69645.304688,67525.422254
2024-04-01,71079.679688,67863.922962


In [3]:


atr_multiple = 1.5
n = 20

df[['avehigh', 'avelow']] = df[['High', 'Low']].rolling(n).mean()
df['avemidprice'] = (df['avehigh'] + df['avelow']) / 2

# get upper and lower bounds to compare to period highs and lows
indicator_atr = ta.volatility.AverageTrueRange(df['High'], df['Low'], df['Close'], window=14, fillna=False)
df['atr14'] = indicator_atr.average_true_range()
df['UPB'] = df['avemidprice'] + atr_multiple * df['atr14']
df['LPB'] = df['avemidprice'] - atr_multiple * df['atr14']

# get the period highs and lows
df['rangemaxprice'] = df[['High']].rolling(n).max()
df['rangeminprice'] = df[['Low']].rolling(n).min()
df['sideways'] = 0

def sideways_range(maxp, minp, upb, lpb):
    if maxp < upb and maxp > lpb and minp < upb and minp > lpb:
        return 1
    else:
        return 0

df['sideways'] = df[['rangemaxprice', 'rangeminprice', 'UPB', 'LPB']].apply(lambda x: sideways_range(x['rangemaxprice'], x['rangeminprice'], x['UPB'], x['LPB']), axis=1)

KeyError: "None of [Index(['High', 'Low'], dtype='object')] are in the [columns]"